# Polynomial interpolation: introduction

This notebook is based on Chapters 6 and 11 of 

<a id="thebook"></a>

> Süli, Endre and Mayers, David F. _An introduction to numerical analysis_. Cambridge University Press, Cambridge, 2003.
<https://doi.org/10.1017/CBO9780511801181> (ebook in [Helka](https://helka.helsinki.fi/permalink/358UOH_INST/1h3k2rg/alma9926836783506253))


We consider the interpolation of a given data set by the polynomial of lowest possible degree that passes through the points of the dataset. Polynomial interpolation, and related techniques such as [Bézier curves](https://en.wikipedia.org/wiki/B%C3%A9zier_curve), can be used to approximate complicated curves given a few points, see the [Utah teapot](https://en.wikipedia.org/wiki/Utah_teapot) for an iconic example. For our purposes, polynomial interpolation forms the basis for algorithms in numerical integration and numerical ordinary differential equations.

# Lagrange interpolation

Let $n \ge 0$ be an integer, define

$$
\mathbb P_n = \{p : \mathbb R \to \mathbb R : \text{$p$ is a polynomial of degree $\le n$} \},
$$

and consider the problem

> Let $x_i, y_i \in \mathbb R$, $i=0,\dots,n$, and suppose that $x_i \ne x_j$ for $i \ne j$.
>
> Find $p \in \mathbb P_n$ such that $p(x_i) = y_i$.

## Theorem: Lagrange interpolation
> For integer $n \ge 1$ and distinct $x_i \in \mathbb R$, $i=0,\dots,n$, the polynomials
>
>$$
L_k(x) = \prod_{i=0, i \ne k}^n \frac{x-x_i}{x_k-x_i}, \qquad k=0,\dots,n,
$$
>
> satisfy $L_k(x_i) = \delta_{ik}$ for all $i,k = 0,\dots,n$. 
> Moreover, for any $y_i \in \mathbb R$, $i = 0,\dots,n$,
>$$
p(x) = \sum_{k=0}^n y_k L_k(x)
$$
>
> satisfies $p \in \mathbb P_n$ and $p(x_i) = y_i$ for $i = 0,\dots,n$. Also, if $q \in \mathbb P_n$ and $q(x_i) = y_i$ for $i = 0,\dots,n$, then $q = p$.
> Setting $L_0 = 1$, the same holds also for $n=0$. 

For a proof, see Lemma 6.1 and Theorem 6.1 in [the book](#thebook).

Let $k \ge 0$ be an interger and let $a < b$. We write $C^k(a,b)$ for the set of real valued functions that are continuous on $[a,b]$ and have continuous derivatives up to order $k$ on $[a,b]$.

## Theorem: error in Lagrange interpolation
> Let $x_0, \dots x_n \in [a,b]$ be distinct, let $f \in C^{n+1}(a,b)$, and set
>
>$$
p(x) = \sum_{k=0}^n f(x_k) L_k(x).
$$
>
> Then for all $x \in [a,b]$ there is $\xi \in (a,b)$ such that 
>
>$$
f(x) - p(x) = \frac{f^{(n+1)}(\xi)}{(n+1)!} \prod_{i=0}^n (x-x_i).
$$

For a proof, see Theorem 6.2 in [the book](#thebook).

# Runge phenomenon

Let $a = -5$, $b = 5$ and consider the Lagrange interpolation polynomial $p$ of the function 

$$
f(x) = \frac{1}{1+x^2}
$$

with equally spaced points on $[a,b]$ 

$$
x_i = a + \frac{i(b-a)}{n}, \qquad i = 0,\dots,n.
$$

Let us find the maximum of $|f - p|$ on $[a,b]$. The maximum is at a critical point of $p - f$ (this sign is more convenient) or at one of the end points $a$ and $b$. A critical point $x$ satisfies $p'(x) - f'(x) = 0$ or equivalently the polynomial equation

$$
(x^2 + 1)^2 p'(x) + 2x = 0.
$$

Let's solve this equation using the polynomial root finding algorithm of NumPy.

In [ ]:
import numpy as np
import scipy.interpolate as interp

def f(x):
    return 1/(1 + x**2)

def p(n):
    '''Lagrange interpolation polynomial of f on [-5,5]'''
    xs = np.linspace(-5, 5, n+1)
    ys = f(xs)
    return interp.lagrange(xs, ys)    

def crit(p):
    '''Critical points of p - f on [-5, 5]'''
    q = np.poly1d([1, 0, 2, 0, 1]) # (x^2 + 1)^2 = x^4 + 2 x^2 + 1
    r = np.poly1d([2, 0]) # 2x
    # The left-hand side of the equation, i.e. q p' + r
    lhs = np.polyadd(np.polymul(q, np.polyder(p)), r)
    zs = np.roots(lhs)
    # Select the real roots 
    ixs = np.abs(np.imag(zs)) < np.finfo(float).eps
    xs = np.real(zs[ixs])
    # Select the roots in (-5,5)
    ixs = np.abs(xs) < 5
    return xs[ixs]

In [ ]:
def dist(p):
    '''Maximum of |f-p| on [-5,5]'''
    xs = crit(p)
    # Append the end points
    xs = np.append(xs, [-5, 5])
    # polyval evaluates a polynomial using Horner's method 
    return np.max(np.abs(f(xs) - np.polyval(p, xs)))

ns = [2**n for n in range(1,6)]
ds = [dist(p(n)) for n in ns]
import pandas as pd
df = pd.DataFrame(ds)
df.columns = ['Max error']
df.index = ns
df.index.name = 'n'
df.style.format('{:.2f}')

In [ ]:
# Plot f and p with n = 10
import matplotlib.pyplot as plt
xs = np.linspace(-5, 5, 100)
plt.plot(xs, f(xs), 'b')
plt.plot(xs, np.polyval(p(10), xs), 'r');

Both $f$ and $p$ extend to the complex plane, however, $f$ is singular at $z = \pm \imath$. (Here $\imath = \sqrt{-1}$.) Let $C$ be a positively oriented simple closed curve in the complex plane and suppose that the line segment $[-5, 5]$ is inside $C$ and that $\pm \imath$ are both outside $C$. Define 

$$
g(z) = \frac{f(z)}{z - x} \prod_{i=0}^n \frac{x - x_i}{z - x_i}.
$$

Then the [residue theorem](https://en.wikipedia.org/wiki/Residue_theorem) implies that 

$$
\frac{1}{2 \pi \imath} \oint_C g(z) dz =  \mathop{\rm Res}(g, x) + \sum_{k=0}^n \mathop{\rm Res}(g, x_k) = f(x) -  \sum_{k=0}^n f(x_k) L_k(x) = f(x) - p(x).
$$

The curve $C$ must intersect the line segment $\{\imath y : |y| < 1 \}$ on the imaginary axis since $\pm \imath$ are outside $C$. Let $z$ be a point in the intersection and let $x$ be close to $-5$. In general, the function $g$ is not small near $z$ since $|x - x_i|$ is larger than $|z - x_i|$ for $x_i$ near $5$.

# Linear interpolating splines

Instead of using a global polynomial approximation like Lagrange interpolation on a large interval, it is often better to divide the interval into small subintervals and look for a piecewise polynomial approximation. 

Let $a < b$ and let $m \ge 1$ be an integer. Let 

$$
a = x_0 < x_1 < \dots < x_m = b.
$$ 

The _linear spline_ interpolating $f$ is 

$$
s(x) = \frac{x - x_{i-1}}{x_i - x_{i-1}} f(x_i) + \frac{x_{i} - x}{x_i - x_{i-1}} f(x_{i-1}), \qquad x \in [x_{i-1}, x_i], \quad i = 1,\dots,m.
$$

We see that $s(x_i) = f(x_i)$ for all $i=0,\dots,m$, $s$ is continuous, and that $s$ a polynomial of first order on each subinterval.
Also, if $m = 1$, then $s \in \mathbb P_1$ is the Lagrange interpolation polynomial of $f$ with $x_0 = a$ and $x_1 = b$.

We write for $f \in C(a,b)$ 

$$
\|f\|_\infty = \max_{x \in [a,b]} |f(x)|.
$$

## Theorem: linear interpolation error
> Let $f \in C^2(a,b)$ and let $s$ be the linear spline interpolating $f$ at 
>
>$$
a = x_0 < x_1 < \dots < x_m = b.
$$
>
> Write $h = \max_{i=1,\dots,m} |x_i - x_{i-1}|$. Then 
>$$
\|f - s\|_\infty \le \frac18 \|h^2 f''\|_\infty.
$$

For a proof, see Theorem 11.1 in [the book](#thebook).

In [ ]:
# Linear spline interpolation of exp(-3x) on [0,1]
def f(x):
    return np.exp(-3*x)

xs = np.linspace(0, 1, 4)
ys = f(xs)
s = interp.make_interp_spline(xs, ys, k=1)

xs_fine = np.linspace(0, 1, 100)
plt.plot(xs_fine, f(xs_fine), 'b')
# s is a function and can be evaluated in the fine grid
plt.plot(xs_fine, s(xs_fine), 'r'); 

# Differentiation

Let $p(x) = \sum_{k=0}^n f(x_k) L_k(x)$ be the Lagrange interpolation polynomial of a function $f$,
with distinct $x_0, \dots, x_n \in [a,b]$. Then $p'$ is an approximation of $f'$. We write $\partial$ for the derivative with respect to $x$.

## Theorem: error in differentiation
> Let $f \in C^{n+1}(a,b)$ and let $p \in \mathbb P_n$ be the Lagrange interpolation polynomial as above. Then there are $\eta_1,\dots,\eta_n \in (a,b)$ such that for all $x \in [a,b]$ there is $\xi \in (a,b)$ such that 
> 
>$$
f'(x) - p'(x) = \frac{f^{(n+1)}(\xi)}{n!} \prod_{i=1}^n (x-\eta_i).
$$
>
> Also, writing $h = b - a$, there is $C>0$, independent of $f$ and $h$, such that
>
>$$
\|h\partial(f - p)\|_\infty \le C \|(h\partial)^{n+1} f\|_\infty.
$$
>

For a proof, see Theorem 6.5 in [the book](#thebook).

## Example: first order finite differences

Let $a = t$ and $b = t + h$ for some $t \in \mathbb R$ and $h > 0$, and let $p \in \mathbb P_1$ be the Lagrange interpolation polynomial of a function $f$ with $x_0 = a$ and $x_1 = b$.
Then 

\begin{align*}
p(x) &= \frac{x - t}{h} f(t+h) + \frac{t + h - x}{h} f(t),
\\
p'(x) &= \frac{f(t+h) - f(t)}{h}.
\end{align*}

Note that $p'$ does not depend on $x$. It is called the [forward finite difference](https://en.wikipedia.org/wiki/Finite_difference) of $f$ at $t$. 

Let $a = t - h/2$ and $b = t + h/2$, and let $p \in \mathbb P_1$ be as above. Then

\begin{align*}
p(x) &= \frac{x - t + h/2}{h} f(t+h/2) + \frac{t + h/2 - x}{h} f(t-h/2),
\\
p'(x) &= \frac{f(t+h/2) - f(t - h/2)}{h},
\end{align*}

and $p'$ is called the central finite difference.

## Example: differentiation is nonetheless hard

Consider the central finite difference of a function $f$ at $t = 0$,
and suppose that $f(\pm h/2)$ is known only up to an additive error $\epsilon_\pm$.
Then we can not compute the correct finite difference, but only

$$
\frac{f(h/2) + \epsilon_+ - f(-h/2) - \epsilon_-}{h}
= \frac{f(h/2) - f(-h/2)}{h} + \frac{\epsilon_+ - \epsilon_-}{h}.
$$

This diverges as $h \to 0$ unless $\epsilon_+ = \epsilon_-$. In practice, errors $\epsilon_\pm$ of order of the machine epsilon are unavoidable, and choosing small $h$ will amplify these. 

Mitigating errors like this is discussed further in the inverse problems courses at UH.

# On the interpolation sub-package of SciPy

We have already seen [make_interp_spline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.make_interp_spline.html) that computes the coefficients of an interpolating B-spline. B-splines are discussed further in Section 11.6 of [the book](#thebook). We will next consider briefly cubic splines, see Sections 11.4-5. 

For more details on interpolation with SciPy see the [tutorial](https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html).

## Example: cubic splines

Let $a = x_0 < x_1 < \dots < x_m = b$.
The _natural cubic spline_ $s$ interpolating a function on $f$ is defined via the conditions

1. $s(x_i) = f(x_i)$ for $i = 0,\dots,m$,
2. $s \in C^2(a,b)$ and $s|_{[x_{i-1},x_i]} \in \mathbb P_3$ for $i=1,\dots,m$,
3. $s''(x_0) = s''(x_m) = 0$.

The _Hermitian cubic spline_ $s$ interpolating a function on $f$ is defined via the conditions

1. $s(x_i) = f(x_i)$ and $s'(x_i) = f'(x_i)$ for $i = 0,\dots,m$,
2. $s \in C^1(a,b)$ and $s|_{[x_{i-1},x_i]} \in \mathbb P_3$ for $i=1,\dots,m$.

Let us plot the error $f - s$ in approximations of 

$$
f(x) = \frac{1}{1+x^2}
$$

on the interval $[0,5]$ by using the natural and Hermite cubic splines $s$ with 

$$
x_0 = 0, \quad x_1 = \frac53, \quad x_2 = \frac{10}3, \quad x_3 = 5.
$$

In [ ]:
def f(x):
    return 1/(1+x**2)
def fprime(x):
    '''The derivative of f'''
    return -2*x/(1+x**2)**2

xs = np.linspace(0, 5, 4)
ys = f(xs)
dys = fprime(xs)

In [ ]:
plt.plot([0,5], [0,0], 'k') # zero level for comparison
xs_fine = np.linspace(0, 5, 100)

# The natural cubic spline
s_nat = interp.make_interp_spline(xs, ys, k=3, bc_type='natural')
plt.plot(xs_fine, s_nat(xs_fine) - f(xs_fine), 'r')

# The Hermite cubic spline
s_her = interp.CubicHermiteSpline(xs, ys, dys)
plt.plot(xs_fine, s_her(xs_fine) - f(xs_fine), 'g');

Clearly the green curve is closer to zero on $[x_1,x_3]$. That is, the Hermite cubic spline gives a better approximation than the natural one. On the other hand, its computation requires also the knowledge of $f'(x_i)$, $i=0,\dots,3$. (A similar plot can be found in [the book](#thebook), see Fig. 11.4.) 